In [111]:
from neo4j import GraphDatabase, basic_auth
import nxneo4j
import os

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth("neo4j", "Kaffe1234"))

In [112]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("basal_ganglia_csv")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [113]:
# Deleting all data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            DETACH DELETE n
        ''')
        print("wipeddatabase")

wipeddatabase


In [114]:
# Adding CellTypes to graph with Cell label
csv_file_path = "file:///%s" % get_csv_path("cell_types.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Cell {id: row.ID, name: row.Cell_type_name, ont:row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Cell", session)

Added 107 nodes of type Cell


In [115]:
# Adding CellTypes to graph
cell_types = [("Neuron", "SAO:1417703748"), 
              ("Oligodendrocyte","SAO:844118448	"), 
              ("Microglia","SAO:789292116"), 
              ("Astrocyte","SAO:1394521419")
             ]
with driver.session() as session:
    for cell_name, cell_ont in cell_types:
        q = "CREATE (:CellType {{ name: '{name}', ont: '{ont}' }})".format(name=cell_name, ont=cell_ont)
        session.run(q )
        
    get_count_of_type("CellType", session)

Added 4 nodes of type CellType


In [116]:
# Adding cell phenotypes
csv_file_path = "file:///%s" % get_csv_path("cell_phenotypes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellPhenotype {id: row.ID, name: row.Phenotype, ont:row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_type("CellPhenotype", session)

Added 107 nodes of type CellPhenotype


In [117]:
# Adding cell phenotype categories
csv_file_path = "file:///%s" % get_csv_path("cell_phenotype_categories.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellPhenotypeCategory {id:row.ID, name: row.Phenotype_category})
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_type("CellPhenotypeCategory", session)

Added 7 nodes of type CellPhenotypeCategory


In [118]:
# Relationship between CellPhenotype and CellPhenotypeCategory
csv_file_path = "file:///%s" % get_csv_path("cell_phenotypes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellPhenotype {id: row.ID})
        MATCH (c:CellPhenotypeCategory {id: row.Phenotype_category })
        MERGE (a)-[:IS_A]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("IS_A", session)

Added 210 relationships of type IS_A


In [119]:
# Relationship between Cell and CellPhenotype
csv_file_path = "file:///%s" % get_csv_path("celltype_phenotype.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Cell {id: row.CellID})
        MATCH (c:CellPhenotype {id: row.PhenotypeID })
        MERGE (a)-[:PHENOTYPE]->(c)
    """ % csv_file_path
with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PHENOTYPE", session)

Added 322 relationships of type PHENOTYPE
